<a href="https://colab.research.google.com/github/zishan7030/AdBuss/blob/main/Copy_of_Copy_of_delhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Business Advisor for opening new Business***


In [ ]:
!pip install beautifulsoup4

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import pandas as pd
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
from folium import * # plotting library
from bs4 import BeautifulSoup
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


**getting requred dataframe of List of neighbourhoods in Delhi. scraping web page**

In [ ]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Delhi").text
soup = BeautifulSoup(req, 'lxml')
div = soup.find('div', class_="mw-parser-output")
print("web Page Imported")
#Code to extract the relevent data from the request object using beautiful soup
data = pd.DataFrame(columns=['Borough','Neighborhood'])
i=-1
flag = False
no=0
prev_borough = None
for child in div.children:
    if child.name:
        span = child.find('span')
        if span!=-1 and span is not None:
            try:
                if span['class'][0] == 'mw-headline' and child.a.text!='edit':
                    prev_borough = child.a.text
                    i+=1
                    flag = True
                    continue
            except KeyError:
                continue
        if child.name=='ul' and flag==True:
            neighborhood = []
            for ch in child.children:
                
                try:
                    data.loc[no]=[prev_borough,ch.text]
                    no+=1
                except AttributeError:
                    pass
        flag = False
data[50:60]
dfn= pd.DataFrame(data)
dfn.shape

web Page Imported


(114, 2)

In [ ]:
dfn.to_csv('/content/delhi_csv.csv', index=False)

Now,We Getting Location Data From Geolocator by providing List_of_neighbourhoods_in_Delhi data



In [ ]:
lat_lng = pd.DataFrame(columns=['latitude','longitude'])
geolocator = Nominatim(user_agent="ny_explorer")
for i in range(114):
    address = data['Neighborhood'].loc[i]+',Delhi'
    try: 
        location = geolocator.geocode(address)
        lat_lng.loc[i]=[location.latitude,location.longitude]
    except AttributeError:
        continue
df1 = data
df2 = lat_lng
delhi_neighbourhood_data = pd.concat([df1, df2], axis=1)

In [ ]:
delhi_neighbourhood_data.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.614193,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Begum Pur,28.725503,77.058371
3,North West Delhi,Karala,28.735140,77.032511
4,North West Delhi,Model Town,28.702714,77.193991


In [ ]:
delhi_neighbourhood_data.shape


(114, 4)

In [ ]:
delhi_neighbourhood_data.to_csv('/content/delhi_lag_lat_csv.csv', index=False)


In [ ]:

df1=pd.read_csv('/content/delhi_csv.csv')
df1.shape

(114, 2)

In [ ]:
df1.head()


,Borough,Neighborhood
0,North West Delhi,Adarsh Nagar
1,North West Delhi,Ashok Vihar
2,North West Delhi,Begum Pur
3,North West Delhi,Karala
4,North West Delhi,Model Town


# **Analysis of neighbourhoods using map**
Creating Folium Map of List of neighbourhoods in Delhi

In [ ]:

import folium
latitude = df2.loc[0, 'latitude']
longitude = df2.loc[0, 'longitude']
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map

for lat, lng, label in zip(df2['latitude'],df2['longitude'],df1['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_delhi)  
    
map_delhi 



Getting Data Using Foursquare API by providing ID and Secret.

In [ ]:

CLIENT_ID = 'MGMBVNJKXHKVKCPQNXRD401BWMBZ4YQUF3JDHG5P3FEANLHR' # your Foursquare ID
CLIENT_SECRET = '10RLP4RHEHTAEXBIOOEP3NV1YYZQA3VMLVGDW3N1UDSQOFTA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MGMBVNJKXHKVKCPQNXRD401BWMBZ4YQUF3JDHG5P3FEANLHR
CLIENT_SECRET:10RLP4RHEHTAEXBIOOEP3NV1YYZQA3VMLVGDW3N1UDSQOFTA


In [ ]:
latitude = df2.loc[0, 'latitude']
longitude = df2.loc[0, 'longitude']
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [ ]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
from pandas.io.json import json_normalize

In [ ]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Bikanerwala,Indian Restaurant,28.613391,77.076084
1,Uttam nagar,Metro Station,28.620201,77.068709
2,Gold's Gym A Block Janakpuri,Gym,28.622439,77.069348
3,Potholes at Dabri,Pool,28.605309,77.072504


In [ ]:

nearby_venues.shape

(4, 4)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
  

In [ ]:
delhi_venues = getNearbyVenues(names =df1 ['Neighborhood'],
                               latitudes =df2 ['latitude'],
                               longitudes =df2 ['longitude'])

Adarsh Nagar
Ashok Vihar
Begum Pur
Karala
Model Town
Narela
Pitam Pura
Rohini Sub City
Shalimar Bagh
Civil Lines
Gulabi Bagh
Kamla Nagar
Kashmiri Gate
Kotwali
Mori Gate; Delhi
Sadar Bazaar
Sarai Rohilla
Shakti Nagar
Tis Hazari
Timarpur
Wazirabad
Dilshad Garden
Naveen Shahdara
New Usmanpur
Shahdara
Shastri Park
Yamuna Vihar
Chandni Chowk
Daryaganj
Jhandewalan
Karol Bagh
Shastri Nagar
Kishanganj
Paharganj
Rajender Nagar
Barakhamba Road
Chanakyapuri
Connaught Place
East Vinod Nagar
Jhilmil Colony
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Preet Vihar
Anand Vihar
Vasundhara Enclave
Vishwas Nagar
Vivek Vihar
Alaknanda
Chattarpur
Chittaranjan Park
Defence Colony
East of Kailash
Govindpuri
Greater Kailash
Green Park
Gulmohar Park
Hauz Khas
Hauz Khas Village
Jangpura
Jor Bagh
Kalkaji
Khanpur
Lajpat Nagar
Lodi Colony
Malviya Nagar
Mehrauli
Neeti Bagh
Nehru Place
Netaji Nagar
New Friends Colony
Nizamuddin West
Okhla
Safdarjung Enclave
Sainik Farm
Saket
Sangam Vihar
Sarita Vihar
Sarojini Nagar
Sarvoday

In [ ]:
print(delhi_venues.shape)
delhi_venues.head()

(2233, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Adarsh Nagar,28.614193,77.071541,Uttam nagar,28.620201,77.068709,Metro Station
2,Adarsh Nagar,28.614193,77.071541,Gold's Gym A Block Janakpuri,28.622439,77.069348,Gym
3,Adarsh Nagar,28.614193,77.071541,Potholes at Dabri,28.605309,77.072504,Pool
4,Ashok Vihar,28.699453,77.184826,Bellagio,28.696361,77.180021,Asian Restaurant


In [ ]:
import pandas as pd

delhi_venues.to_csv('/content/delhi_venues.csv', index=False)

In [ ]:
delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adarsh Nagar,4,4,4,4,4,4
Alaknanda,48,48,48,48,48,48
Anand Vihar,6,6,6,6,6,6
Ashok Nagar,15,15,15,15,15,15
Ashok Vihar,10,10,10,10,10,10
...,...,...,...,...,...,...
Vasundhara Enclave,10,10,10,10,10,10
Vishwas Nagar,28,28,28,28,28,28
Vivek Vihar,7,7,7,7,7,7


In [ ]:
print('There are {} uniques categories.'.format(len(delhi_venues['Venue Category'].unique())))

There are 199 uniques categories.


# Creating Onehot encoding

In [ ]:
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhood'] = delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_onehot.head()

,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,...,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
delhi_onehot.shape

(2233, 199)

In [ ]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped

,Neighborhood,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,...,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Shop,Wings Joint
0,Adarsh Nagar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.000000,0.0,0.020833,0.020833,0.020833,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.020833,0.0,0.0,...,0.0,0.0,0.0,0.0,0.062500,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.020833,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Anand Vihar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ashok Nagar,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.066667,0.0,0.0,...,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ashok Vihar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Vasundhara Enclave,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.1,0.0,0.100000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.100000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,Vishwas Nagar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.03571

# getting Freq of TOP 5 venues.

In [ ]:
num_top_venues = 5

for hood in delhi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = delhi_grouped[delhi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adarsh Nagar----
               venue  freq
0               Pool  0.25
1                Gym  0.25
2  Indian Restaurant  0.25
3      Metro Station  0.25
4      Women's Store  0.00


----Alaknanda----
               venue  freq
0        Coffee Shop  0.10
1  Indian Restaurant  0.08
2        Pizza Place  0.06
3              Hotel  0.06
4         Restaurant  0.06


----Anand Vihar----
           venue  freq
0            Gym  0.33
1    Pizza Place  0.17
2  Big Box Store  0.17
3  Shopping Mall  0.17
4          Hotel  0.17


----Ashok Nagar----
                            venue  freq
0               Indian Restaurant  0.20
1                     Coffee Shop  0.13
2                      Food Truck  0.07
3                           Hotel  0.07
4  Multicuisine Indian Restaurant  0.07


----Ashok Vihar----
               venue  freq
0        Snack Place   0.2
1         Donut Shop   0.1
2  Indian Restaurant   0.1
3     Sandwich Place   0.1
4        Coffee Shop   0.1


----Badarpur----
          

# Getting top 10 venues of all neighbourhoods.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = delhi_grouped['Neighborhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Indian Restaurant,Pool,Gym,Metro Station,Wings Joint,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
1,Alaknanda,Coffee Shop,Indian Restaurant,Pizza Place,Sandwich Place,Restaurant,Hotel,Dessert Shop,Café,Italian Restaurant,Convenience Store
2,Anand Vihar,Gym,Pizza Place,Shopping Mall,Big Box Store,Hotel,Wings Joint,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
3,Ashok Nagar,Indian Restaurant,Coffee Shop,Fast Food Restaurant,ATM,Department Store,Pizza Place,Food Truck,Restaurant,Hotel,Multicuisine Indian Restaurant
4,Ashok Vihar,Snack Place,Fast Food Restaurant,Asian Restaurant,Indian Restaurant,Pizza Place,Donut Shop,South Indian Restaurant,Sandwich Place,Coffee Shop,Food Court
...,...,...,...,...,...,...,...,...,...,...,...
103,Vasundhara Enclave,Indian Restaurant,Pool,Diner,Metro Station,Athletics & Sports,Asian Restaurant,Track,BBQ Joint,Shop & Service,Juice Bar
104,Vishwas Nagar,Coffee Shop,Restaurant,Indian Restaurant,Market,BBQ Joint,Convenience Store,Asian Restaurant,Dessert Shop,Chinese Restaurant,Café
105,Vivek Vihar,Indian Restaurant,Food Truck,Farm,Resort,Restaurant,Café,Wings Joint,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop
106,Wazirabad,Indian Restaurant,Diner,Pizza Place,Shop & Service,Light Rail Station,Metro Station,Wings Joint,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint


# Lets Do clustering For neighbourhoods and venue 5 Cluster

In [ ]:
from sklearn.cluster import KMeans

kclusters = 5

delhi_grouped_clustering = delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 2, 1, 3, 3, 3, 3, 4, 3], dtype=int32)

In [ ]:
neighborhoods_venues_sorted['Neighborhood']


0            Adarsh Nagar
1               Alaknanda
2             Anand Vihar
3             Ashok Nagar
4             Ashok Vihar
              ...        
103    Vasundhara Enclave
104         Vishwas Nagar
105           Vivek Vihar
106             Wazirabad
107          Yamuna Vihar
Name: Neighborhood, Length: 108, dtype: object

In [ ]:
neighborhoods_venues_sorted.head()


In [ ]:
delhi_merged = neighborhoods_venues_sorted

# add clustering labels
delhi_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
delhi_merged.to_csv('/content/delhi_master.csv', index=False)
delhi_merged.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Adarsh Nagar,Indian Restaurant,Pool,Gym,Metro Station,Wings Joint,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,1
1,Alaknanda,Coffee Shop,Indian Restaurant,Pizza Place,Sandwich Place,Restaurant,Hotel,Dessert Shop,Café,Italian Restaurant,Convenience Store,2
2,Anand Vihar,Gym,Pizza Place,Shopping Mall,Big Box Store,Hotel,Wings Joint,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,1
3,Ashok Nagar,Indian Restaurant,Coffee Shop,Fast Food Restaurant,ATM,Department Store,Pizza Place,Food Truck,Restaurant,Hotel,Multicuisine Indian Restaurant,3
4,Ashok Vihar,Snack Place,Fast Food Restaurant,Asian Restaurant,Indian Restaurant,Pizza Place,Donut Shop,South Indian Restaurant,Sandwich Place,Coffee Shop,Food Court,2



delhi_sorted# Appending Lag Lat dataframe to above dataset

In [ ]:
lag_lat=pd.read_csv("//content/delhi_csv.csv")                      
dflag=df2['latitude']                                
dflat=df2['longitude']
lag_lat= pd.concat([dflag, dflat], axis=1)
#lag_lat.head()
delhi_sorted=pd.concat([neighborhoods_venues_sorted, lag_lat], axis=1)
delhi_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,latitude,longitude
0,Adarsh Nagar,Indian Restaurant,Pool,Gym,Metro Station,Wings Joint,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,1.0,28.614193,77.071541
1,Alaknanda,Coffee Shop,Indian Restaurant,Pizza Place,Sandwich Place,Restaurant,Hotel,Dessert Shop,Café,Italian Restaurant,Convenience Store,3.0,28.699453,77.184826
2,Anand Vihar,Gym,Pizza Place,Shopping Mall,Big Box Store,Hotel,Wings Joint,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,2.0,28.725503,77.058371
3,Ashok Nagar,Indian Restaurant,Coffee Shop,Fast Food Restaurant,ATM,Department Store,Pizza Place,Food Truck,Restaurant,Hotel,Multicuisine Indian Restaurant,1.0,28.735140,77.032511
4,Ashok Vihar,Snack Place,Fast Food Restaurant,Asian Restaurant,Indian Restaurant,Pizza Place,Donut Shop,South Indian Restaurant,Sandwich Place,Coffee Shop,Food Court,3.0,28.702714,77.193991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.645112,77.123933
110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.634636,77.107112
111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.639650,77.094039
112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.644009,77.054470


In [ ]:
delhi_sorted.shape

(114, 14)


# **# DATA Analysis start**

# Creating Folium Map of Cluster we just define.

In [ ]:
#map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
import matplotlib.cm as cm
from matplotlib import colors
latitude = delhi_sorted.loc[0, 'latitude']
longitude = delhi_sorted.loc[0, 'longitude']
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, poi,cluster in zip(df2['latitude'],df2['longitude'],df1['Neighborhood'],delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)  
    
map_clusters


delhi_sorted# Show Data of common venue from each cluster

## *first* *cluster* *i*.*e* '*cluster* *1*'

In [ ]:
clus1 = delhi_merged.loc[delhi_merged['Cluster Labels'] == 0, delhi_merged.columns[[1] + list(range(2, delhi_merged.shape[1]))]]
clus1

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
32,ATM,Clothing Store,Mobile Phone Shop,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,0
54,Department Store,Gift Shop,Gym,Italian Restaurant,ATM,Afghan Restaurant,Golf Course,Gastropub,Garden Center,Garden,0
59,Electronics Store,Men's Store,Wings Joint,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,0
60,ATM,Food Truck,Wings Joint,Flower Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,0
65,ATM,Pharmacy,Platform,Vegetarian / Vegan Restaurant,Mobile Phone Shop,Men's Store,Light Rail Station,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,0
72,Pool,Business Service,Music Store,Wings Joint,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,0
82,Flower Shop,Mobile Phone Shop,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,0
93,ATM,Weight Loss Center,Pizza Place,Mobile Phone Shop,Wings Joint,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,0
95,ATM,Historic Site,Castle,Wings Joint,Food & Drink Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,0
97,Movie Theater,Department Store,Chinese Restaurant,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,0


## *second* *cluster* *i*.*e* '*cluster* *2*'

In [ ]:
clus2 = delhi_merged.loc[delhi_merged['Cluster Labels'] == 1 , delhi_merged.columns[[1] + list(range(2, delhi_merged.shape[1]))]]
clus2 


,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Indian Restaurant,Pool,Gym,Metro Station,Wings Joint,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,1
3,Indian Restaurant,Coffee Shop,Fast Food Restaurant,ATM,Department Store,Pizza Place,Food Truck,Restaurant,Hotel,Multicuisine Indian Restaurant,1
10,Indian Restaurant,Hotel,Hostel,Soccer Stadium,History Museum,Metro Station,Movie Theater,Flea Market,Restaurant,Road,1
31,Indian Restaurant,Chinese Restaurant,Japanese Restaurant,Park,Middle Eastern Restaurant,Racetrack,Food Court,Modern European Restaurant,Historic Site,History Museum,1
36,Pharmacy,Indian Restaurant,Plaza,Trail,Flea Market,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,1
40,Indian Restaurant,Hotel,Snack Place,Market,Dessert Shop,Fast Food Restaurant,Coffee Shop,Food & Drink Shop,Miscellaneous Shop,Restaurant,1
53,Indian Restaurant,Snack Place,Fast Food Restaurant,Dessert Shop,Market,Historic Site,Boutique,Food & Drink Shop,American Restaurant,Pizza Place,1
64,Indian Restaurant,Playground,Arcade,Asian Restaurant,Coffee Shop,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,1
73,Indian Restaurant,Pizza Place,Bus Station,Convenience Store,Light Rail Station,Metro Station,Snack Place,Food,Furniture / Home Store,Frozen Yogurt Shop,1
78,Indian Restaurant,Convenience Store,Gym,Restaurant,Wings Joint,Flower Shop,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,1


## *Third* *cluster* *i*.*e* '*cluster* *3*'

In [ ]:
clus3 = delhi_merged.loc[delhi_merged['Cluster Labels'] == 2, delhi_merged.columns[[1] + list(range(2, delhi_merged.shape[1]))]]
clus3

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Gym,Pizza Place,Shopping Mall,Big Box Store,Hotel,Wings Joint,Flower Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,2
16,Pizza Place,Hotel,Snack Place,Indian Restaurant,BBQ Joint,Burrito Place,Light Rail Station,Food & Drink Shop,Fast Food Restaurant,High School,2
17,Pizza Place,Fast Food Restaurant,Dessert Shop,Convenience Store,Sandwich Place,Restaurant,Flea Market,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,2
18,Dessert Shop,Light Rail Station,Hotel,Pizza Place,Indian Chinese Restaurant,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,2
20,Vegetarian / Vegan Restaurant,Park,Light Rail Station,Luggage Store,Wings Joint,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,2
21,Hotel,Indian Restaurant,Hotel Bar,Coffee Shop,Thai Restaurant,Food Court,Road,Shopping Mall,Diner,Metro Station,2
33,Fast Food Restaurant,Indian Restaurant,Market,Park,Convenience Store,Restaurant,Sandwich Place,Dessert Shop,Pizza Place,Hostel,2
34,Indian Restaurant,Hotel,Coffee Shop,Fast Food Restaurant,Dessert Shop,Pizza Place,Snack Place,Motorcycle Shop,BBQ Joint,Gym,2
38,Italian Restaurant,Hotel,Pizza Place,Fast Food Restaurant,Flea Market,Café,Bus Station,Metro Station,Historic Site,Light Rail Station,2
39,Pizza Place,Train Station,Hotel,Light Rail Station,Wings Joint,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,2


## *fourth* *cluster* *i*.*e* '*cluster* *4*'

In [ ]:
clus4 = delhi_merged.loc[delhi_merged['Cluster Labels'] == 3 , delhi_merged.columns[[1] + list(range(2, delhi_merged.shape[1]))]]
clus4

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
1,Coffee Shop,Indian Restaurant,Pizza Place,Sandwich Place,Restaurant,Hotel,Dessert Shop,Café,Italian Restaurant,Convenience Store,3
4,Snack Place,Fast Food Restaurant,Asian Restaurant,Indian Restaurant,Pizza Place,Donut Shop,South Indian Restaurant,Sandwich Place,Coffee Shop,Food Court,3
5,Indian Restaurant,Café,Hotel,Snack Place,Bakery,Deli / Bodega,Plaza,College Gym,Salad Place,French Restaurant,3
6,Train Station,Indian Restaurant,Moving Target,Metro Station,Light Rail Station,Café,Wings Joint,Food,Furniture / Home Store,Frozen Yogurt Shop,3
7,Indian Restaurant,Café,Bar,Chinese Restaurant,Hotel,Coffee Shop,Lounge,Fast Food Restaurant,Pub,Historic Site,3
...,...,...,...,...,...,...,...,...,...,...,...
100,Historic Site,Shopping Mall,Cafeteria,Coffee Shop,Café,Wings Joint,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,3
101,Metro Station,Pizza Place,Asian Restaurant,Donut Shop,Fast Food Restaurant,Café,Moving Target,Fried Chicken Joint,Furniture / Home Store,Shopping Mall,3
102,Metro Station,Pizza Place,Asian Restaurant,Indian Restaurant,Donut Shop,Music Venue,Fast Food Restaurant,Café,Moving Target,Fried Chicken Joint,3
103,Indian Restaurant,Pool,Diner,Metro Station,Athletics & Sports,Asian Restaurant,Track,BBQ Joint,Shop & Service,Juice Bar,3


## *Fifth* *cluster* *i*.*e* '*cluster* *5*'

In [ ]:
clus5 = delhi_merged.loc[delhi_merged['Cluster Labels'] == 4, delhi_merged.columns[[1] + list(range(2, delhi_merged.shape[1]))]]
clus5

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
8,Accessories Store,Business Service,Wings Joint,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,4


# **Analysis result:**

# Lets Finding Best location or Neighborhood for new Business.

In [ ]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped

,Neighborhood,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,...,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Weight Loss Center,Whisky Bar,Wine Shop,Wings Joint
0,Adarsh Nagar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.000000,0.0,0.020833,0.020833,0.020833,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.020833,0.0,0.0,...,0.0,0.0,0.0,0.0,0.062500,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.020833,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Anand Vihar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ashok Nagar,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.066667,0.0,0.0,...,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ashok Vihar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Vasundhara Enclave,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.1,0.0,0.100000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.100000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,Vishwas Nagar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.03571

In [ ]:
len(delhi_grouped[delhi_grouped["Coffee Shop"] > 0])
delhi_rest = delhi_grouped[["Neighborhood","Coffee Shop"]]
delhi_rest


,Neighborhood,Coffee Shop
0,Adarsh Nagar,0.000000
1,Alaknanda,0.104167
2,Anand Vihar,0.000000
3,Ashok Nagar,0.133333
4,Ashok Vihar,0.100000
...,...,...
103,Vasundhara Enclave,0.000000
104,Vishwas Nagar,0.107143
105,Vivek Vihar,0.000000
106,Wazirabad,0.000000


# **Clustering venue**

In [ ]:

kclusters = 5

delhi_clustering = delhi_rest.drop(["Neighborhood"], 1)
delhi_clustering.head()

# run k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=12).fit(delhi_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 3, 2, 1, 1, 4, 1, 1], dtype=int32)


### **Getting Mean for each Neighborhoods of coffee shop.**

In [ ]:
merged = delhi_rest.copy()

# add clustering labels
merged["Category"] = kmeans.labels_
#merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
merged.head()

,Neighborhood,Coffee Shop,Category
0,Adarsh Nagar,0.000000,1
1,Alaknanda,0.104167,2
2,Anand Vihar,0.000000,1
3,Ashok Nagar,0.133333,3
4,Ashok Vihar,0.100000,2


### **Create Folium Map to analys cluster**

In [ ]:
lag_lat=pd.read_csv("//content/delhi_csv.csv")
dflag=df2['latitude']
dflat=df2['longitude']
lag_lat= pd.concat([dflag, dflat], axis=1)
#lag_lat.head()
delhi_w_Loc=pd.concat([merged, lag_lat], axis=1)
delhi_w_Loc.head()

,Neighborhood,Coffee Shop,Category,latitude,longitude
0,Adarsh Nagar,0.000000,1.0,28.614193,77.071541
1,Alaknanda,0.104167,2.0,28.699453,77.184826
2,Anand Vihar,0.000000,1.0,28.725503,77.058371
3,Ashok Nagar,0.133333,3.0,28.735140,77.032511
4,Ashok Vihar,0.100000,2.0,28.702714,77.193991


In [ ]:
#map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
import matplotlib.cm as cm
from matplotlib import colors
latitude = delhi_w_Loc.loc[0, 'latitude']
longitude = delhi_w_Loc.loc[0, 'longitude']
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to map
for lat, lng, poi,cluster in zip(df2['latitude'],df2['longitude'],df1['Neighborhood'],merged['Category']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster -1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)  
    
map_clusters

In [ ]:
map_clusters.to_csv("map")

**find value of k Elbow method**

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
%matplotlib inline
elbo=pd.read_csv("income.csv")

plt.scatter(df.Age,df['Income($)'])
plt.xlabel('Age')
plt.ylabel('Income($)')


In [ ]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 0]
#Sort
delhi_w_Loc.sort_values(["Category"], inplace=True, ascending=False)
delhi_w_Loc

,Neighborhood,Coffee Shop,Category,latitude,longitude
22,East Vinod Nagar,0.200000,4.0,28.677339,77.286016
100,Tughlaqabad,0.200000,4.0,28.560691,77.160791
34,Jhandewalan,0.103448,3.0,NaN,NaN
49,Malviya Nagar,0.115385,3.0,28.497140,77.181251
26,Greater Kailash,0.097561,3.0,28.700372,77.272773
...,...,...,...,...,...
109,NaN,NaN,NaN,28.645112,77.123933
110,NaN,NaN,NaN,28.634636,77.107112
111,NaN,NaN,NaN,28.639650,77.094039
112,NaN,NaN,NaN,28.644009,77.054470


Category 1: Neighborhoods with very low number of Coffee Shop

In [ ]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 0.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
78,Rohini Sub City,0.000000,0.0,28.574157,77.195370
105,Vivek Vihar,0.000000,0.0,28.657859,77.142429
77,Rama Krishna Puram,0.000000,0.0,28.528574,77.288331
70,Pandav Nagar,0.000000,0.0,28.567101,77.269764
76,Rajender Nagar,0.000000,0.0,28.605127,76.999492
...,...,...,...,...,...
44,Kotwali,0.000000,0.0,28.641115,77.312502
43,Kishanganj,0.015152,0.0,28.641441,77.295259
42,Kirti Nagar,0.000000,0.0,28.617955,77.286747
41,Khanpur,0.000000,0.0,28.613350,77.295538



Category 2: Neighborhoods with low number of coffee shop

In [ ]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 1.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
88,Sarvodaya Enclave,0.130435,1.0,28.593833,77.134979
64,New Friends Colony,0.142857,1.0,28.590702,77.220921
3,Ashok Nagar,0.133333,1.0,28.735140,77.032511
27,Green Park,0.151515,1.0,28.656097,77.235436
56,Munirka,0.153846,1.0,28.557101,77.213005
92,Shastri Nagar,0.166667,1.0,28.667367,77.184438
86,Sarita Vihar,0.153846,1.0,28.610770,77.091106


In [ ]:

delhi_w_Loc.loc[delhi_w_Loc['Category'] == 2.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
80,Safdarjung Enclave,0.061728,2.0,28.552146,77.224698
24,Fateh Nagar,0.058824,2.0,28.673333,77.289025
40,Kashmiri Gate,0.035714,2.0,28.630553,77.277575
37,Kalkaji,0.052632,2.0,28.631383,77.219792
68,Paharganj,0.046512,2.0,28.549257,77.252952
28,Gulabi Bagh,0.064516,2.0,28.646090,77.243048
55,Moti Nagar,0.060606,2.0,28.555537,77.202497
58,Naraina Vihar,0.062500,2.0,28.553855,77.194713
85,Sarai Rohilla,0.058824,2.0,28.493170,77.303024
69,Palam,0.062500,2.0,28.573534,77.186359


In [ ]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 3.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
34,Jhandewalan,0.103448,3.0,NaN,NaN
49,Malviya Nagar,0.115385,3.0,28.497140,77.181251
26,Greater Kailash,0.097561,3.0,28.700372,77.272773
29,Gulmohar Park,0.089744,3.0,28.644319,77.199917
35,Jhilmil Colony,0.083333,3.0,28.629318,77.225841
99,Tis Hazari,0.090909,3.0,28.529249,77.154134
45,Lajpat Nagar,0.084211,3.0,28.601726,77.321122
25,Govindpuri,0.094118,3.0,28.668434,77.250530
51,Mehrauli,0.085714,3.0,28.571791,77.232010
52,Model Town,0.090909,3.0,28.557032,77.244614


In [ ]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 4.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
22,East Vinod Nagar,0.2,4.0,28.677339,77.286016
100,Tughlaqabad,0.2,4.0,28.560691,77.160791


## **Making CSV files of all clusters**

In [ ]:
clus1.to_csv('/content/clus1.csv', index=False)

In [ ]:
clus2.to_csv('/content/clus2.csv', index=False)

In [ ]:
clus3.to_csv('/content/clus3.csv', index=False)

In [ ]:
clus4.to_csv('/content/clus4.csv', index=False)

In [ ]:
clus5.to_csv('/content/clus5.csv', index=False)

In [ ]:

import pandas


# 1 cluster
clus1=pd.read_csv('/content/clus1.csv')
count1=0
for i in range(len(clus1)) : 
  #value of j after checking below condition either True or False
  j1=clus1.loc[i,"1st Most Common Venue"] =='Indian Restaurant' or clus1.loc[i,"2nd Most Common Venue"] =='Indian Restaurant' or clus1.loc[i,"3rd Most Common Venue"] =='Indian Restaurant' or clus1.loc[i,"4th Most Common Venue"] =='Indian Restaurant' or clus1.loc[i,"5th Most Common Venue"] =='Indian Restaurant' 
  if j1==True: 
   count1=count1+1
#print(count1)


# 2 cluster
clus2=pd.read_csv('/content/clus2.csv')
count2=0
for i in range(len(clus2)) :
  #value of j after checking below condition either True or False
  j2=clus2.loc[i,"1st Most Common Venue"] =='Indian Restaurant' or clus2.loc[i,"2nd Most Common Venue"] =='Indian Restaurant' or clus2.loc[i,"3rd Most Common Venue"] =='Indian Restaurant' or clus2.loc[i,"4th Most Common Venue"] =='Indian Restaurant' or clus2.loc[i,"5th Most Common Venue"] =='Indian Restaurant' 
  if j2==True:
    count2=count2+1
#print(count2)


# 3 cluster
clus3=pd.read_csv('/content/clus3.csv')
count3=0
for i in range(len(clus3)) :
  #value of j after checking below condition either True or False
  j3=clus3.loc[i,"1st Most Common Venue"] =='Indian Restaurant' or clus3.loc[i,"2nd Most Common Venue"] =='Indian Restaurant' or clus3.loc[i,"3rd Most Common Venue"] =='Indian Restaurant' or clus3.loc[i,"4th Most Common Venue"] =='Indian Restaurant' or clus3.loc[i,"5th Most Common Venue"] =='Indian Restaurant' 
  if j3==True:
    count3=count3+1
#print(count3)


# 4 cluster
clus4=pd.read_csv('/content/clus4.csv')
count4=0
for i in range(len(clus4)) :
  #value of j after checking below condition either True or False
  j4=clus4.loc[i,"1st Most Common Venue"] =='Indian Restaurant' or clus4.loc[i,"2nd Most Common Venue"] =='Indian Restaurant' or clus4.loc[i,"3rd Most Common Venue"] =='Indian Restaurant' or clus4.loc[i,"4th Most Common Venue"] =='Indian Restaurant' or clus4.loc[i,"5th Most Common Venue"] =='Indian Restaurant' 
  if j4==True:
    count4=count4+1
#print(count4)

# 5 cluster
clus5=pd.read_csv('/content/clus5.csv')
count5=0
for i in range(len(clus5)) :
  #value of j after checking below condition either True or False
  j5=clus5.loc[i,"1st Most Common Venue"] =='Indian Restaurant' or clus5.loc[i,"2nd Most Common Venue"] =='Indian Restaurant' or clus5.loc[i,"3rd Most Common Venue"] =='Indian Restaurant' or clus5.loc[i,"4th Most Common Venue"] =='Indian Restaurant' or clus5.loc[i,"5th Most Common Venue"] =='Indian Restaurant' 
  if j5==True:
    count5=count5+1
#print(count5)



# ploting on map 
total_counts = [count1, count2, count3,count4,count5]
cnt=[i for i in range(len(total_counts)) if total_counts[i]==0] 
#print(cnt)

for x in cnt:
  if x == 0 or x==1 or x==2 or x==3 or x==4:
    if x==0:
      print('cluster1')
      plot1= delhi_sorted.loc[delhi_sorted["Cluster Labels"]==0,'Neighborhood']
      plot1=pd.DataFrame(plot1)
      print(plot1)
    
    if x==1:
      print('cluster2')
      plot2= delhi_sorted.loc[delhi_sorted["Cluster Labels"]==1,'Neighborhood']
      plot2=pd.DataFrame(plot2)
      print(plot2)
     
    if x==2:
      print('cluster3')
      plot3= delhi_sorted.loc[delhi_sorted["Cluster Labels"]==2,'Neighborhood']
      plot3=pd.DataFrame(plot3)
      print(plot3)
    
      
    if x==3:
      print('cluster4')
      plot4= delhi_sorted.loc[delhi_sorted["Cluster Labels"]==3,'Neighborhood']
      plot4=pd.DataFrame(plot4)
      print(plot4)
       
    
    if x==4:
      print('cluster5')
      plot5= delhi_sorted.loc[delhi_sorted["Cluster Labels"]==4,'Neighborhood']
      plot5=pd.DataFrame(plot5)
      print(plot5)
    x = x+1
  else:
    print('no place for business')

 

cluster1
       Neighborhood
32        Inderpuri
54        Moti Bagh
59           Narela
60  Naveen Shahdara
65     New Usmanpur
72      Patel Nagar
82      Sainik Farm
93     Shastri Park
95  South Extension
97  Sundar Nagar[1]
98         Timarpur
cluster5
  Neighborhood
8    Begum Pur
